	# N-Gram Model to Generate Text
    ### Enter an input to make it sound like Kafka and Dante!

In [1]:
import numpy as np
import random
import re

In [2]:
%pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vsita_ru1\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [3]:
import anvil.server

# anvil.server.connect('client_FQMN4IG2MJ7K74OVJKCGJVZD-6RKH5345YDHTXRDQ')
anvil.server.connect('server_LABXEYN6RHARVQE75RZMVEWB-6RKH5345YDHTXRDQ')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [4]:
def generate_ngrams(text, n):
    words = re.findall(r'\b\w+\b', text)
    ngrams = []
    for i in range(len(words) - n + 1):
        ngrams.append(tuple(words[i:i+n]))
    return ngrams

In [5]:
def train_model(text, n):
    ngrams = generate_ngrams(text, n)
    model = {}
    for ngram in ngrams:
        prefix = ngram[:-1]
        suffix = ngram[-1]
        if prefix not in model:
            model[prefix] = {}
        if suffix not in model[prefix]:
            model[prefix][suffix] = 0
        model[prefix][suffix] += 1
    return model

In [6]:
# def laplace_smoothing(model, n, prefix):
#     count = sum(model[prefix].values()) if prefix in model else 0
#     return (count + 1) / (sum(sum(counts.values()) for counts in model.values()) + len(model))

In [7]:
def laplace_smoothing(model, vocabulary_size, k):
    smoothed_model = {}
    for prefix in model:
        smoothed_model[prefix] = {}
        prefix_count = sum(model[prefix].values())
        for suffix in model[prefix]:
            smoothed_model[prefix][suffix] = (model[prefix][suffix] + k) / (prefix_count + k * vocabulary_size)
    return smoothed_model

In [8]:
def generate_text(model, n, seed_text, max_length):
    words = seed_text.split()
    prefix = tuple(words[-n+1:])
    output = seed_text
    while len(words) < max_length:
        if prefix in model:
            suffixes = model[prefix]
            next_word = random.choices(list(suffixes.keys()), weights=list(suffixes.values()), k=1)[0]
        else:
            next_word = random.choice(list(model.keys()))
        output += ' ' + str(next_word)
        words.append(next_word)
        prefix = tuple(words[-n+1:])
    return output


In [9]:
with open('../resources/kafka-metamorphosis.txt', 'r', encoding='utf-8') as f:
        text = f.read()
vocabulary_size = len(set(text.split()))
model = train_model(text, 3)
smoothed_model = laplace_smoothing(model, vocabulary_size, 1)

In [10]:
import anvil.media

@anvil.server.callable
def generate_result(userinput):
    generated_text = generate_text(smoothed_model, 3, userinput, 100)
    return generated_text

In [11]:
anvil.server.wait_forever()